# Group 20 — Exploratory Data Analysis

This notebook performs exploratory data analysis on customer and flights databases, covering data inspection, quality assessment, and preprocessing.

## Table of Contents
- [Data Import](#data-import)
- [Data Inspection](#data-inspection)
  - [Customer DB](#customer-db)
  - [Flights DB](#flights-db)
- [Missing Values](#missing-values)
- [Outliers](#outliers)
  - [Customer DB Outliers](#customer-outliers)
  - [Flights DB Outliers](#flights-outliers)
- [Correlations](#correlations)
- [Miscellaneous Analyses](#miscellaneous)
  - [Categorical Distributions](#categorical-distributions)
  - [Relationship Analysis](#relationship-analysis)
  - [Points Redemption Analysis](#points-redemption)
  - [Geospatial Analysis](#geospatial-analysis)
- [Data Preprocessing](#preprocessing)

# <a id="data-import"></a> Data Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from scipy import stats
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# Load the data

customer_db = pd.read_csv("data/DM_AIAI_CustomerDB.csv", index_col=0 )
flights_db = pd.read_csv("data/DM_AIAI_FlightsDB.csv")


# <a id="data-inspection"></a> Data Inspection

### <a id="customer-db"></a> Customer DB

## Basic Information

In [ ]:
customer_db.head()

In [ ]:
customer_db.info()

In [ ]:
# Check for duplicate loyalty IDs
duplicated_loyalty_ids = customer_db[customer_db['Loyalty#'].duplicated()]['Loyalty#'].unique()
print(f"Number of unique Duplicated Loyalty IDs: {len(duplicated_loyalty_ids)}")

## <a id="flights-db"></a> Flights DB

In [ ]:
flights_db.head()

In [ ]:
flights_db.info()

In [ ]:
# Check for fractional flight counts
invalid_fractional_flights = flights_db[
    (flights_db['NumFlights'] % 1 != 0) |
    (flights_db['NumFlightsWithCompanions'] % 1 != 0)
]

print(f"Number of rows with fractional flight counts: {len(invalid_fractional_flights)}")
if not invalid_fractional_flights.empty:
    display(invalid_fractional_flights[['Year', 'Month', 'NumFlights', 'NumFlightsWithCompanions']].head(10))

In [ ]:
# Check for inconsistent flight records (NumFlights=0 but DistanceKM>0)
invalid_flights = flights_db[(flights_db['NumFlights'] == 0) & (flights_db['DistanceKM'] > 0)]

print(f"Number of inconsistent rows (NumFlights=0 & DistanceKM>0): {len(invalid_flights)}")
if not invalid_flights.empty:
    display(invalid_flights.head())

# <a id="missing-values"></a> Missing Values Analysis

In [ ]:
def missing_report(df: pd.DataFrame) -> pd.DataFrame:
    out = df.isna().agg(['sum', 'mean']).T
    out.columns = ['Total', 'Percentage']
    out['Percentage'] = (out['Percentage'] * 100).round(2)
    return out.sort_values(['Total', 'Percentage'], ascending=False)

In [ ]:
print("Customer DB Missing Values:")
customer_missing = missing_report(customer_db)
customer_missing

In [ ]:
print("Flights DB Missing Values:")
flights_missing = missing_report(flights_db)
flights_missing

## Conclusion

**Customer DB:** CancellationDate has 86% missing values (expected for active customers). Income and Customer Lifetime Value have minimal missing data (0.12%). All other columns are complete.

**Flights DB:** No missing values detected. Dataset has excellent completeness.

# <a id="outliers"></a> Outliers Analysis

## <a id="customer-outliers"></a> Customer DB Outliers

In [ ]:
numeric_features = ["Income", "Customer Lifetime Value"]

# Histograms
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for ax, feat in zip(axes.flatten(), numeric_features):
    ax.hist(customer_db[feat].dropna(), bins=30)
    ax.set_title(feat)
plt.suptitle("Customer DB - Histograms")
plt.show()

# Boxplots
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for ax, feat in zip(axes.flatten(), numeric_features):
    sns.boxplot(data=customer_db, y=feat, ax=ax)
    ax.set_title(feat)
plt.suptitle("Customer DB - Boxplots")
plt.tight_layout()
plt.show()

## <a id="flights-outliers"></a> Flights DB Outliers

In [ ]:
numeric_features = ["NumFlights", "NumFlightsWithCompanions", "DistanceKM", 
                    "PointsAccumulated", "PointsRedeemed", "DollarCostPointsRedeemed"]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
for ax, feat in zip(axes.flatten(), numeric_features):
    sns.boxplot(data=flights_db, y=feat, ax=ax)
    ax.set_title(feat)
plt.suptitle("Flights DB - Boxplots")
plt.tight_layout()
plt.show()

## Conclusion

**Customer DB:** Both Income and Customer Lifetime Value show right-skewed distributions with numerous outliers. Many customers have zero or near-zero income (likely students), while a small segment has very high income.

**Flights DB:** All flight-related metrics exhibit right skewness with outliers representing highly active frequent flyers. These outliers are legitimate and represent valuable customers rather than data errors.

# <a id="correlations"></a> Correlations Analysis

In [ ]:
numeric_features = ["NumFlights", "NumFlightsWithCompanions", "DistanceKM", 
                    "PointsAccumulated", "PointsRedeemed", "DollarCostPointsRedeemed"]

corr = flights_db[numeric_features].corr(method="pearson")

fig = plt.figure(figsize=(10, 8))
sns.heatmap(data=corr, annot=True, cmap='coolwarm', center=0)
plt.title("Flights DB - Correlation Matrix")
plt.tight_layout()
plt.show()

## Conclusion

| Pair | Correlation | Interpretation |
|------|--------------|----------------|
| NumFlights and DistanceKM | 0.62 | Strong positive — more flights generally means more total distance flown. |
| NumFlights and PointsAccumulated | 0.62 | Strong positive — more flights results in more points earned. |
| NumFlights and NumFlightsWithCompanions | 0.51 | Moderate positive — people who fly often also tend to fly with companions more. |
| NumFlightsWithCompanions and DistanceKM | 0.39 | Moderate — more companion flights slightly increase total distance. |
| PointsRedeemed and DollarCostPointsRedeemed | 1.00 | Perfect correlation — these two represent the same underlying concept (points redeemed vs. their dollar cost).
| PointsRedeemed and NumFlights / DistanceKM / PointsAccumulated | 0.19–0.34 | Weak relationships — redeeming points doesn’t strongly depend on flying behavior in this dataset. |

# <a id="miscellaneous"></a> Miscellaneous Analyses

## <a id="categorical-distributions"></a> Categorical Distributions

In [ ]:
categorical_cols = ['Province or State', 'City', 'Gender', 'Education',
                    'Location Code', 'Marital Status', 'LoyaltyStatus', 'EnrollmentType']

fig, axes = plt.subplots(nrows=math.ceil(len(categorical_cols) / 3), ncols=3, figsize=(15, 15))

for ax, col in zip(axes.flatten(), categorical_cols):
    customer_db[col].value_counts().plot(kind='barh', ax=ax, title=f'Distribution of {col}')
    ax.set_xlabel("Count")
    ax.set_ylabel("")
    ax.invert_yaxis()

plt.tight_layout()
plt.show()

## <a id="relationship-analysis"></a> Relationship Analysis

In [ ]:
# Customer DB: Income vs Customer Lifetime Value
sns.scatterplot(data=customer_db, x='Income', y='Customer Lifetime Value', alpha=0.6)
plt.title('Income vs Customer Lifetime Value')
plt.show()

# Income by Loyalty Status
sns.boxplot(x='LoyaltyStatus', y='Income', data=customer_db.dropna(subset=['Income']))
plt.title('Income by Loyalty Tier')
plt.xlabel('Loyalty Tier')
plt.ylabel('Income')
plt.show()

# Income statistics by Loyalty Status
customer_db.groupby('LoyaltyStatus')['Income'].agg(['count', 'mean', 'median'])

**Income vs CLV:** Weak relationship indicates that customer value is driven by factors beyond income, such as travel frequency and engagement with loyalty programs.

**Income by Loyalty Status:** Income distribution appears similar across loyalty tiers, suggesting the program is accessible to all income levels.

In [ ]:
# ANOVA test: Income vs Education
groups = [group["Income"].dropna() for _, group in customer_db.groupby("Education")]
f_stat, p_val = stats.f_oneway(*groups)
print(f"ANOVA F-statistic: {f_stat:.3f}, p-value: {p_val:.4f}")

**ANOVA Result:** Tests whether income differs significantly across education levels (Bachelor, College, etc.). A low p-value (<0.05) would indicate significant differences in income by education level.

In [ ]:
# Flights DB: Pairplot of key features
numeric_features = ["NumFlights", "NumFlightsWithCompanions", "DistanceKM", 
                    "PointsAccumulated", "PointsRedeemed", "DollarCostPointsRedeemed"]

g = sns.pairplot(data=flights_db[numeric_features].sample(5000, random_state=42),
                 diag_kind='scatter', plot_kws={'alpha': 0.5}, height=2)
plt.suptitle('Flights DB - Feature Relationships', y=1.02)
plt.show()

**Pairplot Analysis:** Visualizes relationships between all flight-related features. Strong diagonal patterns indicate perfect correlations (e.g., DistanceKM vs PointsAccumulated), while scattered plots show weak or no relationships (e.g., redemption vs flight activity).

**Key Insights:**
- No clear linear relationship between Income and CLV
- High variance in CLV across all income levels suggests other factors drive customer value
- Strong positive relationships between flight frequency, distance, and points accumulated
- Redemption behavior appears independent of flight activity

## <a id="points-redemption"></a> Points Redemption Analysis

In [ ]:
points_accumulated = flights_db['PointsAccumulated'].sum()
points_redeemed = flights_db['PointsRedeemed'].sum()
points_non_redeemed = points_accumulated - points_redeemed

fig, ax = plt.subplots(figsize=(10, 8))
sizes = [points_non_redeemed, points_redeemed]
labels = ['Points Non-Redeemed', 'Points Redeemed']
colors = ['#66b3ff', '#ff9999']

ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%',
       wedgeprops={"linewidth": 1, "edgecolor": "white"})
plt.title('Points Redemption Distribution')
plt.axis('equal')
plt.show()

**Points Redemption Overview:** Shows the proportion of loyalty points that have been redeemed vs those still available. High unredeemed percentage suggests opportunity for campaigns to increase engagement.

## <a id="geospatial-analysis"></a> Geospatial Analysis

In [ ]:
# Geographic distribution of customers
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot of customer locations with Canada boundaries
axes[0].scatter(customer_db['Longitude'], customer_db['Latitude'], alpha=0.6, s=15, c='red', edgecolor='darkred')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].set_title('Customer Geographic Distribution Across Canada')
axes[0].set_xlim(-141, -52)  # Canada longitude range
axes[0].set_ylim(41, 70)      # Canada latitude range
axes[0].grid(True, alpha=0.3)

# Province distribution
province_counts = customer_db['Province or State'].value_counts()
axes[1].barh(province_counts.index, province_counts.values, color='steelblue')
axes[1].set_xlabel('Number of Customers')
axes[1].set_title('Customers by Province/State')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print(f"Total unique locations: {customer_db[['Latitude', 'Longitude']].drop_duplicates().shape[0]}")

**Geographic Distribution:** Map shows customer locations concentrated in southern Canada's major urban areas. Eastern provinces (Ontario, Quebec) and western coast (British Columbia) have highest customer density, reflecting population distribution patterns.

# <a id="preprocessing"></a> Data Preprocessing

In this section we apply the preprocessing and feature engineering steps
described in the report:

- Fix data types and logical inconsistencies in **FlightsDB**
- Handle missing values and transform skewed variables in **CustomerDB**
- Create new features: log-transformed variables, points utilisation,
  cancellation flag, customer value score, flight activity score, and
  average flight distance.



In [ ]:
import numpy as np

def preprocess_flights(flights_df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply preprocessing steps to the FlightsDB:
    - Convert YearMonthDate to datetime
    - Round down NumFlights and NumFlightsWithCompanions
    - Set DistanceKM = 0 where NumFlights == 0
    - Drop DollarCostPointsRedeemed
    - Add log-transformed versions of skewed variables
    - Create PointsUtilizationRatio = PointsRedeemed / PointsAccumulated
    """
    df = flights_df.copy()

    # 1. YearMonthDate -> datetime
    if 'YearMonthDate' in df.columns:
        df['YearMonthDate'] = pd.to_datetime(df['YearMonthDate'])

    # 2. Round down flight counts and cast to int
    for col in ['NumFlights', 'NumFlightsWithCompanions']:
        if col in df.columns:
            df[col] = np.floor(df[col]).astype(int)

    # 3. Fix logical inconsistency: DistanceKM must be 0 if NumFlights == 0
    if {'NumFlights', 'DistanceKM'}.issubset(df.columns):
        df.loc[df['NumFlights'] == 0, 'DistanceKM'] = 0

    # 4. Drop perfectly correlated variable
    if 'DollarCostPointsRedeemed' in df.columns:
        df = df.drop(columns=['DollarCostPointsRedeemed'])

    # 5. Log transforms for skewed numeric variables
    log_cols = ['DistanceKM', 'PointsAccumulated', 'PointsRedeemed']
    for col in log_cols:
        if col in df.columns:
            df[f'{col}_log'] = np.log1p(df[col])

    # 6. Points utilisation ratio
    if {'PointsRedeemed', 'PointsAccumulated'}.issubset(df.columns):
        denom = df['PointsAccumulated'].replace({0: np.nan})
        df['PointsUtilizationRatio'] = df['PointsRedeemed'] / denom

    return df


def preprocess_customers(customer_df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply preprocessing steps to the CustomerDB:
    - Create cancellation flag from CancellationDate
    - Group-median imputation (by LoyaltyStatus) for Income and Customer Lifetime Value
    - Log transform Customer Lifetime Value
    - Create Location feature (region) from Province or State (placeholder mapping)
    - Create CustomerValueScore composite feature
    """
    df = customer_df.copy()

    # 1. Cancellation flag
    if 'CancellationDate' in df.columns:
        df['CancelledFlag'] = df['CancellationDate'].notna().astype(int)

    # 2. Group-median imputation by LoyaltyStatus
    group_col = 'LoyaltyStatus'
    cols_to_impute = ['Income', 'Customer Lifetime Value']
    for col in cols_to_impute:
        if col in df.columns and group_col in df.columns:
            df[col] = df.groupby(group_col)[col].transform(
                lambda x: x.fillna(x.median())
            )

    # 3. Log transform Customer Lifetime Value (for variance stabilisation)
    if 'Customer Lifetime Value' in df.columns:
        df['CustomerLifetimeValue_log'] = np.log1p(df['Customer Lifetime Value'])

    # 4. Location feature (region mapping) – fill mapping as desired
    if 'Province or State' in df.columns:
        region_map = {
            # Example mapping – adjust to your data
            # 'Ontario': 'Central',
            # 'Quebec': 'Central',
            # 'British Columbia': 'West',
            # 'Alberta': 'West',
            # 'Nova Scotia': 'East',
            # ...
        }
        df['Location'] = df['Province or State'].map(region_map).fillna(df['Province or State'])

    # 5. Customer Value Score (simple composite of CLV and Income)
    clv_col = 'CustomerLifetimeValue_log'
    if clv_col in df.columns:
        clv_scaled = (df[clv_col] - df[clv_col].mean()) / df[clv_col].std(ddof=0)

        if 'Income' in df.columns:
            income_log = np.log1p(df['Income'].clip(lower=0))
            income_scaled = (income_log - income_log.mean()) / income_log.std(ddof=0)
            # Heavier weight on CLV, lighter on Income
            df['CustomerValueScore'] = 0.7 * clv_scaled + 0.3 * income_scaled
        else:
            df['CustomerValueScore'] = clv_scaled

    return df
 

In [ ]:
def build_customer_flight_features(flights_df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate monthly flight records into customer-level features:
    - TotalFlights, TotalDistanceKM, TotalPointsAccumulated, TotalPointsRedeemed
    - MeanPointsUtilization
    - AverageFlightDistance
    - FlightActivityScore (based on z-scores of flights and distance)
    """
    id_col = 'Loyalty#'
    df = flights_df.copy()

    agg = (
        df
        .groupby(id_col)
        .agg(
            TotalFlights=('NumFlights', 'sum'),
            TotalDistanceKM=('DistanceKM', 'sum'),
            TotalPointsAccumulated=('PointsAccumulated', 'sum'),
            TotalPointsRedeemed=('PointsRedeemed', 'sum'),
            MeanPointsUtilization=('PointsUtilizationRatio', 'mean')
        )
        .reset_index()
    )

    # Average flight distance = total distance / total flights
    agg['AverageFlightDistance'] = agg['TotalDistanceKM'] / agg['TotalFlights'].replace({0: np.nan})

    # FlightActivityScore: combines total flights and distance (z-scores)
    for col in ['TotalFlights', 'TotalDistanceKM']:
        mean = agg[col].mean()
        std = agg[col].std(ddof=0)
        if std == 0:
            agg[f'{col}_z'] = 0
        else:
            agg[f'{col}_z'] = (agg[col] - mean) / std

    agg['FlightActivityScore'] = agg['TotalFlights_z'] + agg['TotalDistanceKM_z']

    return agg


In [ ]:
# Run preprocessing for each table
customer_preprocessed = preprocess_customers(customer_db)
flights_preprocessed = preprocess_flights(flights_db)

# Build customer-level flight features
customer_flight_features = build_customer_flight_features(flights_preprocessed)

# Merge into a single modelling dataset (one row per customer)
id_col = 'Loyalty#'
model_df = (
    customer_preprocessed
    .merge(customer_flight_features, on=id_col, how='left')
)

print("Customer-preprocessed shape:", customer_preprocessed.shape)
print("Flights-preprocessed shape:", flights_preprocessed.shape)
print("Model dataset shape:", model_df.shape)

model_df.head()


In [ ]:

def winsorize_dataframe(df, columns, limits=(0.01, 0.01)):
    """
    Apply winsorization to each column in `columns`.
    limits=(lower_pct, upper_pct) means: cap values at the 1st and 99th percentile.

    Returns the winsorized copy of df.
    """
    df = df.copy()
    for col in columns:
        if col in df.columns:
            # winsorize returns masked arrays -> convert to normal array
            df[col] = winsorize(df[col], limits=limits).data
    return df


In [ ]:
flight_outlier_cols = [
    'TotalFlights',
    'TotalDistanceKM',
    'TotalPointsAccumulated',
    'TotalPointsRedeemed',
    'AverageFlightDistance'
]

customer_flight_features_wins = winsorize_dataframe(
    customer_flight_features,
    columns=flight_outlier_cols,
    limits=(0.01, 0.01)    # winsorize at 1% and 99%
)


In [ ]:
customer_outlier_cols = [
    'Income',
    'Customer Lifetime Value',
    'CustomerLifetimeValue_log'
]

customer_preprocessed_wins = winsorize_dataframe(
    customer_preprocessed,
    columns=customer_outlier_cols,
    limits=(0.01, 0.01)
)


In [ ]:
id_col = 'Loyalty#'

model_df = (
    customer_preprocessed_wins
    .merge(customer_flight_features_wins, on=id_col, how='left')
)

print("Final model_df shape:", model_df.shape)
model_df.head()


In [ ]:
# Check for remaining missing values
model_df.isna().sum().sort_values(ascending=False).head(15)


In [ ]:
# Drop CancellationDate (flag already exists)
model_df = model_df.drop(columns=['CancellationDate'], errors='ignore')

# Fill average distance for zero-flight customers
model_df['AverageFlightDistance'] = model_df['AverageFlightDistance'].fillna(0)

# Fill points utilization for customers with no point activity
model_df['MeanPointsUtilization'] = model_df['MeanPointsUtilization'].fillna(0)

# Fill ALL flight-related NaNs with 0
flight_cols = [
    'TotalFlights', 'TotalDistanceKM', 'TotalPointsAccumulated',
    'TotalPointsRedeemed', 'MeanPointsUtilization', 'AverageFlightDistance',
    'TotalFlights_z', 'TotalDistanceKM_z', 'FlightActivityScore'
]

for col in flight_cols:
    model_df[col] = model_df[col].fillna(0)


In [ ]:
# Numerical features for clustering
numeric_features = [
    'Income',
    'CustomerLifetimeValue_log',
    'CustomerValueScore',
    
    'TotalFlights',
    'TotalDistanceKM',
    'AverageFlightDistance',
    'TotalPointsAccumulated',
    'TotalPointsRedeemed',
    'MeanPointsUtilization',
    'FlightActivityScore',

    'CancelledFlag'
]

# Categorical features to encode
categorical_features = [
    'EnrollmentType',  # Bronze, Silver, Gold ...
    'Location'         # Region/Province after mapping
]


In [ ]:
df_selected = model_df[numeric_features + categorical_features].copy()
df_selected.head()


In [ ]:
df_encoded = pd.get_dummies(df_selected, columns=categorical_features, drop_first=True)
df_encoded.head()


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_encoded)

X_scaled[:5]


# DBSCAN 


In [ ]:
def get_dbscan_outliers(df: pd.DataFrame, features: list, eps=1.9, min_samples=20):
    """
    Apply DBSCAN to identify outliers in the dataset.
    Returns a DataFrame with original data and DBSCAN labels.
    """
    
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
    labels = dbscan.fit_predict(df[features])
    df_out = df.copy()
    df_out['DBSCAN_Label'] = labels
    return df_out

In [ ]:
df_outliers = get_dbscan_outliers(df_selected, numeric_features, eps=1.9, min_samples=20)
df_outliers.head()

In [ ]:
df_outliers['DBSCAN_Label'].unique()

In [ ]:
n_clusters = n_clusters = len(set(labels)) - (1 if -1 in labels.values else 0)
print(f"Estimated number of clusters: {n_clusters}")

 Given the fact that -1 represents the outliers, than there are two clusters (0,1)

In [ ]:
print("\n6. Distribution:")
for label in sorted(labels.unique()):
    count = (labels == label).sum()
    pct = count / len(labels) * 100
    type = "OUTLIERS" if label == -1 else f"Cluster {label}"
    print(f"   {type:15} → {count:5} points ({pct:5.2f}%)")